In [ ]:
from apps.afterme.paper import Paper
from datetime import datetime, timezone
import time
from firebase_admin import initialize_app, firestore, credentials
from web3 import Web3
import os
import sys
import re
import argparse # <<< ADDED: For handling command-line arguments

# --- Argument Parsing to select network ---
parser = argparse.ArgumentParser(description="AfterMe Indexer for a specific Etherlink network.")
parser.add_argument(
    'network',
    choices=['mainnet', 'testnet'], 
    help="The network to run the indexer on ('mainnet' or 'testnet')."
)
args = parser.parse_args(['testnet']) 
# --- End of Argument Parsing ---

# --- Network-specific Configuration ---
if args.network == 'mainnet':
    firestore_doc_name = "Etherlink"
    print("--- CONFIGURATION: MAINNET ---")
elif args.network == 'testnet':
    firestore_doc_name = "Etherlink-Testnet"
    print("--- CONFIGURATION: TESTNET ---")
else:
    # This case is technically handled by argparse's `choices`, but it's good practice
    print(f"FATAL: Invalid network '{args.network}' specified.")
    sys.exit(1)
# --- End of Network Configuration ---

# --- Firebase and Web3 Setup ---
cred = credentials.Certificate('afterme.json')
initialize_app(cred) # <<< MODIFIED: Give each app a unique name to avoid conflicts
db = firestore.client()
networks = db.collection("networks")
ceva = networks.document(firestore_doc_name).get() # <<< MODIFIED: Use variable for doc name
wrapper_address = ceva.to_dict()['sourceContractAddress']
rpc = ceva.to_dict()['rpc']
print(f"RPC Endpoint: {rpc}")
print("Original Wrapper address: " + str(wrapper_address))

web3 = Web3(Web3.HTTPProvider(rpc))
if not web3.is_connected():
    print("FATAL: Node connection failed!")
    sys.exit()
print("Node connected successfully.")
event_signatures = {
    web3.keccak(text="WillCreated(address,address)").hex(): "WillCreated",
    web3.keccak(text="WillCleared(address,address)").hex(): "WillCleared",
    web3.keccak(text="Ping(uint256)").hex(): "Ping",
    web3.keccak(text="Executed(address,uint256,address)").hex(): "Executed",
    web3.keccak(text="Cancelled(uint256)").hex(): "Cancelled",
}

print("--- Initializing with Monitored Event Signatures ---")
for hash_val, name in event_signatures.items():
    print(f"- {name}: {hash_val}")
print("----------------------------------------------------")
papers = {}
daos_collection = db.collection("contracts") 
docs = list(daos_collection.stream())
will_addresses = [doc.id for doc in docs]
listening_to_addresses = [wrapper_address]
listening_to_addresses.extend(will_addresses)
listening_to_addresses = list(set([addr for addr in listening_to_addresses if addr]))



--- CONFIGURATION: TESTNET ---
RPC Endpoint: https://node.ghostnet.etherlink.com
Original Wrapper address: 0x1f734BAf289A308ede3de1F97152c93026f62466
Node connected successfully.


In [ ]:
processed_transactions = set()
heartbeat = 0
while True:
    heartbeat += 1
    try:
        latest = web3.eth.block_number
        first = latest - 15 if latest > 15 else 0 
        
        logs = web3.eth.get_logs({
            "fromBlock": first,
            "toBlock": latest,
            "address": listening_to_addresses,
        })
        if logs:
            print(f"Found {len(logs)} logs between blocks {first} and {latest}") # <<< MODIFIED: Added network context to log
        for log in logs:
            tx_hash = log["transactionHash"].hex()
            if tx_hash in processed_transactions:
                continue 
            processed_transactions.add(log.transactionHash.hex())
            contract_address = Web3.to_checksum_address(log["address"])
            
            if not log["topics"]:
                print(f"Skipping log with no topics: {log}")
                continue

            event_signature_from_log = log["topics"][0].hex()
            event_name = event_signatures.get(event_signature_from_log) 
            
            if not event_name:
                print(f"Unknown event signature: {event_signature_from_log} in log: {log}")
                continue
            print(f"-> Event: {event_name}, Contract: {contract_address}, Tx: {tx_hash}") # <<< MODIFIED: Added network context    
            new_contract_addresses = papers[contract_address].handle_event(log, func=event_name)

    except Exception as e:
        import traceback
        print(f"MAIN LOOP ERROR [{args.network.upper()}]: {e}")
        print(traceback.format_exc())
        try:
            web3 = Web3(Web3.HTTPProvider(rpc))
            if web3.is_connected(): print("Node reconnected successfully.")
            else: print("Node reconnection failed!")
        except Exception as recon_e:
            print(f"Error during reconnection: {recon_e}")
            
    if heartbeat % 50 == 0:
        print(f"[{args.network.upper()}] Heartbeat: {heartbeat}. Listening to {len(listening_to_addresses)} addresses.")

    time.sleep(5)

In [10]:
for log in logs:
    tx_hash = log["transactionHash"].hex()
    if not log["topics"]:
        print(f"Skipping log with no topics: {log}")
        continue

    event_signature_from_log = log["topics"][0].hex()
    event_name = event_signatures.get(event_signature_from_log) 
    
    if not event_name:
        print(f"Unknown event signature: {event_signature_from_log} in log: {log}")
        continue
    print(f"-> Event: {event_name}, Contract: {contract_address}, Tx: {tx_hash}") # <<< MODIFIED: Added network context    
    # new_contract_addresses = papers[contract_address].handle_event(log, func=event_name)


-> Event: WillCreated, Contract: 0x1f734BAf289A308ede3de1F97152c93026f62466, Tx: 1dda4b4fd75f31977f80a9f85673f20d1ea656ced438a8ab46fbd50bcb645df6
